In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import requests
import json


import numpy as np
import pandas as pd
import pickle as pkl



# import torch
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
#     HfArgumentParser,
#     Trainer,
#     TrainingArguments,
# )

import warnings
import random
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1600)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
def load_jsonl_data(file, output_df=True):
    data_list = []
    with open(file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            try:
                data_list.append(json.loads(json_str))
            except:
                pass
    if output_df:
        df = pd.DataFrame.from_dict(data_list)
        # print(len(data_list))
        if len(df)>0:
            return df
        else:
            return None
    else:
        return data_list

In [ ]:
zfill_num = 4
content = []
for i, row in data.iterrows():
    prev_id = None
    zfill_i = str(i).zfill(4)
    row_convo = row['conversations']
    item_num = 0
    zfill_item_num = str(len(row_convo))
    prev_user_txt = ''
    prev_assistant_text = '' 
    latest_user_txt = ''
    latest_assistant_text = '' 

    item_dict = {}
    item_dict['user']=''
    item_dict['assistant']=''    
    for j in range(0, len(row_convo)):
        item_dict['system'] = row['system']
        conv_item = row_convo[j]
        if conv_item['role']=='assistant':
            prev_assistant_text = latest_assistant_text
            latest_assistant_text = conv_item['content']
            item_dict['assistant'] = latest_assistant_text
            
        if conv_item['role']=='user':
            prev_user_text = latest_user_txt
            latest_user_txt = conv_item['content']
            item_dict['user'] = latest_user_txt
                
        
        item_dict['convo_end'] = (j==len(row_convo)-1)
        # item_dict['user_length'] = len(conv_item['user'])
        # # if (item_dict['user_length'] <= 200) |  item_dict['convo_end']:
        # if item_dict['convo_end']:
        # print(item_dict['user'] , item_dict['assistant'])
        user_and_assistant = (len(item_dict['assistant'])>0) & (len(item_dict['user'])>0)
        # print(user_and_assistant)
        add_item_cond = item_dict['convo_end'] | user_and_assistant
        # print(item_dict['convo_end'], user_and_assistant)
        if add_item_cond:
            zfill_item = str(item_num).zfill(len(zfill_item_num))
            conv_id = f'conv{zfill_i}_item{zfill_item}'        
            item_dict['id'] = conv_id
            item_dict['parent_id'] = prev_id        
            item_dict['row_id'] = i
            prev_id = conv_id
            content.append(item_dict)
            item_dict = {}
            item_dict['user']=''
            item_dict['assistant']=''
            item_num +=1

# break
convdf = pd.DataFrame.from_dict(content)

In [ ]:
convdf_sp100 = convdf.copy()
convdf_sp100['system'] = sp100

In [ ]:
convdf_sp100.to_parquet('../feature_data/task2/task2_conversation_sp100.parquet')

In [ ]:
# further clean up

sp100v2 = sp100[sp100['assistant']!=''].reset_index(drop=True)
sp100v2.shape



# checking data

In [2]:
data = pd.read_parquet('../feature_data/task5/task5_conversation_sp100.parquet')

In [3]:
row_idx = np.random.choice(np.arange(data['row_id'].max()), 1)[0]

use_cols = ['user', 'assistant', 'id', 'parent_id', 'row_id']

data[data['row_id']==row_idx][use_cols]

,user,assistant,id,parent_id,row_id
621,What tools are available for Farcaster?,"Farcaster offers several tools such as Hubble, a hub for reading and writing messages, and Replicator, a tool to sync a hub to a postgres database.",conv0245_item0,None,245


In [25]:
data[use_cols].sample(5)

,user,assistant,id,parent_id,row_id
285,What do I need to do to set up my Farcaster profile?,"To set up your Farcaster profile, you need to first create a Farcaster account by calling the IdGateway contract. After that, you'll need to get a username, rent storage, and add a key. These steps involve multiple signatures and onchain transactions.",conv0109_item0,None,109
310,How can I join these developer calls?,You can join the developer calls by adding the [Calendar Invite](https://calendar.google.com/calendar/u/0?cid=NjA5ZWM4Y2IwMmZiMWM2ZDYyMTkzNWM1YWNkZTRlNWExN2YxOWQ2NDU3NTA3MjQwMTk3YmJlZGFjYTQ3MjZlOEBncm91cC5jYWxlbmRhci5nb29nbGUuY29t) to your calendar.,conv0120_item3,conv0120_item2,120
309,Are there any developer calls I can join to discuss the protocol?,"Yes, we host a bi-weekly developer call to discuss upcoming changes to the protocol. The call is open to anyone and is a great way to get involved.",conv0120_item2,conv0120_item1,120
567,What tools are available for developers on Farcaster?,"There are several tools available for developers on Farcaster, including Hubble, a hub for reading and writing messages, and Replicator, a tool to sync a hub to a PostgreSQL database.",conv0223_item0,None,223
502,Where can I find recordings of past developer calls?,Recordings of previous developer calls can be watched on YouTube. The link to the recordings is available in the documentation.,conv0194_item3,conv0194_item2,194


In [4]:
data['system'].values[0]

'You are a helpful assistant who is helping a customer with questions about Farcaster.'

# model validation

In [ ]:
# cd /src

model_path = 'yifanxie/literate-toucanet1-1-1'
apikey = 'E1D0YPIGPWKKWS4AAI6ROZHZ1PDJFH9R'
eval_file = '../llm-loss-validator/src/data/dummy_data.jsonl'
valarg_file = '../llm-loss-validator/src/validation_config_cpu.json.example'

!CUDA_VISIBILE_DEVICES=0 FLOCK_API_KEY={apikey} python ../llm-loss-validator/src/validate.py validate \
--model_name_or_path {model_path} \
--base_model gemma \
--eval_file {eval_file} \
--context_length 128 \
--max_params 4000000000 \
--local_test \
--validation_args_file {valarg_file}

# model submission

In [4]:
apikey = 'E1D0YPIGPWKKWS4AAI6ROZHZ1PDJFH9R'

In [6]:
apikey = 'E1D0YPIGPWKKWS4AAI6ROZHZ1PDJFH9R'
location = 'https://fed-ledger-prod.flock.io/api/v1/tasks/submit-result'
repo = "yifanxie/tacky-cassowary-2-1"
header = {
        "flock-api-key":apikey,
        "'Content-Type": "application/json",
}
data = {
    "task_id":5,
    "data":{
        "hg_repo_id": repo, 
        "base_model": "phi"         
    }
}

response = requests.post(location, headers=header, json=data)
res = json.loads(response.text)
print(res)

{'detail': [{'type': 'enum', 'loc': ['base_model'], 'msg': "Input should be 'llama2', 'llama3', 'qwen1.5', 'yi', 'mistral', 'mixtral', 'gemma' or 'zephyr'", 'input': 'phi', 'ctx': {'expected': "'llama2', 'llama3', 'qwen1.5', 'yi', 'mistral', 'mixtral', 'gemma' or 'zephyr'"}, 'url': 'https://errors.pydantic.dev/2.7/v/enum'}]}


In [ ]:
# curl --location 'https://fed-ledger-prod.flock.io/api/v1/tasks/submit-result' --header 'flock-api-key: E1D0YPIGPWKKWS4AAI6ROZHZ1PDJFH9R' --header 'Content-Type: application/json' --data '{
#     "task_id": 2, 
#     "data":{
#         "hg_repo_id": "yifanxie/literate-toucanet1-1-1", 
#         "base_model": "gemma" 
#     }
# }'